In [ ]:
!pip install torchmetrics matplotlib fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 

In [ ]:
#Redas Laukžemis 2213754
#Parrot Car Cat

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as T
import torchmetrics
from torchmetrics import F1Score
from torchmetrics.segmentation import DiceScore
import os
import fiftyone as fo
import shutil
import torch.optim as optim
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

classes = ["Parrot", "Car", "Cat"]

class_map = {
    "Parrot": 1,
    "Car": 2,
    "Cat": 3
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class SegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths, augment=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.augment = augment

        self.transform = A.Compose([
            A.Resize(256, 256),
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
            A.ColorJitter(p=0.2),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])

        self.no_aug_transform = A.Compose([
            A.Resize(256, 256),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert("RGB")
        mask = Image.open(self.mask_paths[index]).convert("L")

        image = np.array(image)
        mask = np.array(mask)

        transform = self.transform if self.augment else self.no_aug_transform
        transformed = transform(image=image, mask=mask)

        image = transformed["image"]
        mask = transformed["mask"].long()

        return image, mask

class UNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Encoder (downsampling)
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)  # Added deeper layer
        self.pool = nn.MaxPool2d(2)

        # Decoder (upsampling)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(256, 128)  # Skip connection added
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(128, 64)   # Skip connection added
        self.final = nn.Conv2d(64, num_classes, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),  # Added batch norm
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)        # 64 channels
        e2 = self.enc2(self.pool(e1))  # 128 channels
        e3 = self.enc3(self.pool(e2))  # 256 channels (new)

        # Decoder with skip connections
        d3 = self.up3(e3)        # 128 channels
        d3 = torch.cat([d3, e2], dim=1)  # Skip connection
        d3 = self.dec3(d3)       # 128 channels

        d2 = self.up2(d3)        # 64 channels
        d2 = torch.cat([d2, e1], dim=1)  # Skip connection
        d2 = self.dec2(d2)       # 64 channels

        return self.final(d2)

model = UNet(len(class_map) + 1).to(device)

In [ ]:
def create_masks_dir(dataset, class_name, class_map, image_dir, mask_dir):
    class_id = class_map[class_name]

    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)

    for sample in dataset:
        img_w, img_h = Image.open(sample.filepath).size

        # Save image
        image = Image.open(sample.filepath)
        filename = os.path.basename(sample.filepath)
        image.save(os.path.join(image_dir, filename))

        # Full mask (shared between classes)
        full_mask_path = os.path.join(mask_dir, f"{os.path.splitext(filename)[0]}.png")
        if os.path.exists(full_mask_path):
            full_mask_np = np.array(Image.open(full_mask_path))
        else:
            full_mask_np = np.zeros((img_h, img_w), dtype="uint8")

        for det in sample.ground_truth.detections:
            if det.label == class_name:
                # Original mask
                mask = (det.mask * 255).astype("uint8")
                mask_img = Image.fromarray(mask).convert("L")

                x, y, w, h = det.bounding_box
                x1, y1 = int(x * img_w), int(y * img_h)
                x2, y2 = int((x + w) * img_w), int((y + h) * img_h)

                mask_resized = mask_img.resize((x2 - x1, y2 - y1), resample=Image.NEAREST)
                mask_array = np.array(mask_resized)

                # Set pixels to class_id where mask > 0
                mask_region = full_mask_np[y1:y2, x1:x2]
                mask_region[mask_array > 0] = class_id
                full_mask_np[y1:y2, x1:x2] = mask_region

        # Save updated full mask
        full_mask = Image.fromarray(full_mask_np)
        full_mask.save(full_mask_path)

In [ ]:
def createDataset(className, split, size):
    return fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split=split,
    label_types="segmentations",
    classes=[className],
    max_samples=size,
    only_matching=True,
    overwrite=True
)

In [ ]:
def get_image_mask_paths(image_dir, mask_dir):
    image_paths = []
    mask_paths = []

    # Get list of image files (assuming they're all .jpg or .png)
    image_filenames = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))]

    for filename in image_filenames:
        image_path = os.path.join(image_dir, filename)

        # The mask has the same filename (except extension) as the image
        mask_filename = os.path.splitext(filename)[0] + ".png"
        mask_path = os.path.join(mask_dir, mask_filename)

        if os.path.exists(mask_path):
            image_paths.append(image_path)
            mask_paths.append(mask_path)

    return image_paths, mask_paths

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-5):
        super().__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = F.softmax(preds, dim=1)
        targets_onehot = F.one_hot(targets, num_classes=preds.shape[1]).permute(0, 3, 1, 2).float()
        intersection = (preds * targets_onehot).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets_onehot.sum(dim=(2, 3))
        dice = (2 * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

In [ ]:
def train(model, dataloader, num_epochs=50):
    model.train()

    weights = torch.tensor([0.2, 1.0, 1.0, 1.0]).to(device)
    loss = DiceLoss() + nn.CrossEntropyLoss(weight=weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, step_size=10, gamma=0.5)



    for epoch in range(num_epochs):
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model(images)
            loss = loss(preds, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
def test(model, loader):
    model.eval()

    dice_metric = DiceScore(num_classes=4, average='macro', include_background=False, input_format='index').to('cpu')
    f1_macro = F1Score(task='multiclass', num_classes=4, average='macro').to('cpu')
    f1_micro = F1Score(task='multiclass', num_classes=4, average='micro').to('cpu')

    model.to(device)

    with torch.inference_mode():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.squeeze(1).to(device)

            preds = model(images)
            preds = torch.argmax(preds, dim=1)

            dice_metric.update(preds.cpu(), masks.cpu())
            f1_macro.update(preds.cpu(), masks.cpu())
            f1_micro.update(preds.cpu(), masks.cpu())


    dice_score = dice_metric.compute()
    f1_macro_score = f1_macro.compute()
    f1_micro_score = f1_micro.compute()

    print(f"Dice Score: {dice_score}")
    print(f"F1 Macro Score: {f1_macro_score}")
    print(f"F1 Micro Score: {f1_micro_score}")

In [ ]:
train_image_dir = "/train_images"
train_mask_dir = "/train_masks"

if os.path.exists(train_image_dir):
    shutil.rmtree(train_image_dir)
if os.path.exists(train_mask_dir):
    shutil.rmtree(train_mask_dir)

for class_name in classes:
    dataset = createDataset(class_name, "train", 300)
    create_masks_dir(dataset, class_name, class_map, train_image_dir, train_mask_dir)
    dataset.delete()

image_paths, mask_paths = get_image_mask_paths(train_image_dir, train_mask_dir)

Overwriting existing directory '/root/fiftyone/open-images-v7/train'


INFO:fiftyone.zoo.datasets:Overwriting existing directory '/root/fiftyone/open-images-v7/train'


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [5.0s elapsed, 0s remaining, 1.5Gb/s]         


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'train'


 100% |█████████████████| 300/300 [11.3s elapsed, 0s remaining, 18.9 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 300/300 [11.3s elapsed, 0s remaining, 18.9 samples/s]      


Dataset 'open-images-v7-train-300' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7-train-300' created


In [ ]:
train_dataset = SegmentationDataset(image_paths, mask_paths, augment=True)

sample_img, sample_mask = train_dataset[0]
print("Unique values in mask:", torch.unique(sample_mask))
print("Mask value counts:", torch.bincount(sample_mask.flatten()))

import matplotlib.pyplot as plt

# def verify_dataset(dataset):
#     for i in range(min(5, len(dataset))):  # Check first 5 samples
#         img, mask = dataset[i]
#         print(f"\nSample {i}:")
#         print(f"Image shape: {img.shape}, Mask shape: {mask.shape}")
#         print(f"Unique mask values: {torch.unique(mask)}")
#         print(f"Pixel counts: {torch.bincount(mask.flatten())}")

#         # Visual check
#         plt.figure(figsize=(12,4))
#         plt.subplot(131); plt.imshow(img.permute(1,2,0)); plt.title("Image")
#         plt.subplot(132); plt.imshow(mask); plt.title("Mask")
#         plt.subplot(133); plt.imshow(img.permute(1,2,0))
#         plt.imshow(mask, alpha=0.3); plt.title("Overlay")
#         plt.show()

# verify_dataset(train_dataset)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, prefetch_factor=2)

train(model, train_dataloader)

Unique values in mask: tensor([0, 3])
Mask value counts: tensor([13269,     0,     0, 52267])


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 1, Loss: 0.9783
Epoch 2, Loss: 0.6713
Epoch 3, Loss: 0.8846
Epoch 4, Loss: 0.6799
Epoch 5, Loss: 0.7846
Epoch 6, Loss: 0.5759
Epoch 7, Loss: 1.0783
Epoch 8, Loss: 1.4020
Epoch 9, Loss: 0.7321
Epoch 10, Loss: 0.7470
Epoch 11, Loss: 1.1627
Epoch 12, Loss: 0.8798
Epoch 13, Loss: 1.4037
Epoch 14, Loss: 0.9643
Epoch 15, Loss: 1.2174
Epoch 16, Loss: 0.8676
Epoch 17, Loss: 0.7232
Epoch 18, Loss: 1.1133
Epoch 19, Loss: 0.9888
Epoch 20, Loss: 0.7131
Epoch 21, Loss: 1.1073
Epoch 22, Loss: 0.7556
Epoch 23, Loss: 0.9017
Epoch 24, Loss: 0.6367
Epoch 25, Loss: 0.8098
Epoch 26, Loss: 0.8299
Epoch 27, Loss: 0.8123
Epoch 28, Loss: 1.3093
Epoch 29, Loss: 0.9398
Epoch 30, Loss: 0.7809
Epoch 31, Loss: 0.5433
Epoch 32, Loss: 0.8800
Epoch 33, Loss: 0.7754
Epoch 34, Loss: 0.8844
Epoch 35, Loss: 1.3349
Epoch 36, Loss: 0.6101
Epoch 37, Loss: 1.8195
Epoch 38, Loss: 0.7991
Epoch 39, Loss: 0.8710
Epoch 40, Loss: 0.8979
Epoch 41, Loss: 1.2266
Epoch 42, Loss: 0.8835
Epoch 43, Loss: 0.5425
Epoch 44, Loss: 0.56

In [ ]:
test_image_dir = "/test_images"
test_mask_dir = "/test_masks"

if os.path.exists(test_image_dir):
    shutil.rmtree(test_image_dir)
if os.path.exists(test_mask_dir):
    shutil.rmtree(test_mask_dir)

for class_name in classes:
    dataset = createDataset(class_name, "validation", 75)
    create_masks_dir(dataset, class_name, class_map, test_image_dir, test_mask_dir)
    dataset.delete()

test_images, test_masks = get_image_mask_paths(test_image_dir, test_mask_dir)

Overwriting existing directory '/root/fiftyone/open-images-v7/validation'


INFO:fiftyone.zoo.datasets:Overwriting existing directory '/root/fiftyone/open-images-v7/validation'


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/open-images-v7/validation' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/validation/validation-images-with-rotation.csv' to '/root/fiftyone/open-images-v7/validation/metadata/image_ids.csv'


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/validation/metadata/classes.csv'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 75/75 [3.0s elapsed, 0s remaining, 24.4 samples/s]      


INFO:eta.core.utils: 100% |███████████████████| 75/75 [3.0s elapsed, 0s remaining, 24.4 samples/s]      


Dataset 'open-images-v7-validation-75' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7-validation-75' created


In [ ]:
test_dataset = SegmentationDataset(test_images, test_masks, augment=False)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2, prefetch_factor=2)

test(model, test_dataloader)

torch.save(model.state_dict(), "unet_multiclass.pth")

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Dice Score: 0.1672876924276352
F1 Macro Score: 0.5257026553153992
F1 Micro Score: 0.6180934309959412
